In [6]:
import os
import pandas as pd
import time
import re
import tqdm
from ast import literal_eval

In [7]:
PATH = './data/sorted/python/prepared'
REPOS = os.listdir(PATH)
DEV = True
REMOVE_ONE_LETTER = True


if DEV: 
    f = 'DeepLearning'

In [8]:
df = pd.read_csv('./data/python_quantlets.csv', converters={'Sourcecode': literal_eval})

In [4]:
df = pd.read_csv('./data/python_quantlets.csv', converters={'Sourcecode': literal_eval})
df = df.explode('Sourcecode')
df = df[df.Sourcecode!='BEGINNING OF CODE']
df['Sourcecode'] = df['Sourcecode'].apply(lambda x: ' '.join(x))
df = df.reset_index(drop=True)

if DEV:
    df = df[~df.Sourcecode.str.startswith('{')]

In [5]:
def parse_imports(code):
    pattern_import = re.compile('(?<=import\s)\w+')
    matches_import = pattern_import.findall(code)

    pattern_from = re.compile('(?<=from\s)[\w|.]+(?=\simport)')
    matches_from = pattern_from.findall(df.Sourcecode[0])

    matches_import.extend(matches_from)
    return ' '.join(matches_import)


def parse_func_name(code):
    pattern = re.compile('(?<=def\s)[\w|.]+(?=\()')
    matches = pattern.findall(code)
    return ' '.join(matches)


def parse_func_args(code, one_letter_removal=True):
    pattern = re.compile('(?<=def).+(?=\))')
    matches = pattern.findall(code)
    final_matches = ''
    for match in matches:
        final_matches += match.split('(')[1]
    final_matches = final_matches.replace(',', ' ')

    if one_letter_removal:
        new_matches = []
        for word in final_matches.split(' '):
            if len(word)>1:
                new_matches.append(word)

        final_matches = ' '.join(new_matches)

    #print(new_matches)

    return final_matches

def parse_vars(code):
    pass

def parse_all(code):
    pattern = re.compile('(?<=import\s)\w+|(?<=from\s)[\w|.]+(?=\simport)|(?<=def\s).+(?=\))|(?<=#).+')
    matches = pattern.findall(code)
    new_matches = []
    for match in matches:
        if '(' in match:
            new_matches.extend(match.split('('))
        else:
            new_matches.append(match)
        
    return ' '.join(new_matches).replace(',', ' ')

def parse_comments(code):
    pattern = re.compile('(?<=#).+')#|(?<=\"\"\")(?s).+(?=\"\"\")
    matches = pattern.findall(code)
    return matches

def parse_docstrings(code):
    pattern = re.compile('(?<=\"\"\")(?s).+?(?=\"\"\")')
    matches = pattern.findall(code)
    return matches
    


In [6]:
df['ParsedSource'] = df.Sourcecode.apply(parse_all)

In [7]:
df['num_parsed_code_tokens'] = df['ParsedSource'].apply(lambda x: len(x.split()))

In [21]:
df.to_csv('data/parsed_source_code_df.csv', index=False)

In [ ]:
# TODOS:

# remove ------- from comments
# remove ##### from comments
# remove one letter variables
# remove 
# split Camel Cases
# split on _ 
# split on .

In [ ]:
##### ENCODE THE DESCRIPTION

In [ ]:
##### ENCODE THE CODE

In [ ]:
##### ENCODE THE KEYWORDS